In [1]:
import numpy as np
from bmcs_shell.api import MATS2DElastic, FETriangularMesh, TriXDomainFE
from ibvpy.bcond import BCDof
from ibvpy.sim.tstep_bc import TStepBC

# Uniaxial tension test

In [2]:
wb_mesh = FETriangularMesh(
    X_Id = [[0,0,0], [1000,0,0], [1000,1000,0],[0,1000,0]],
    I_Fi = [[0,1,2], [0,2,3]],
)
fixed_dofs = [0,1,2,5,8,9,11]
loaded_dofs = [3,6]

In [3]:
xdomain = TriXDomainFE(
    integ_factor = 10,
    mesh=wb_mesh
)

In [4]:
tmodel = MATS2DElastic(E_=28000, nu=0.2)

In [5]:
u_cntl = 1000
bc_fixed = [BCDof(var= 'u', dof=dof, value=0 )
           for dof in fixed_dofs]
bc_loaded = [BCDof(var= 'f', dof=dof, value=u_cntl )
           for dof in loaded_dofs]

In [6]:
ts = TStepBC(
    domains=[(xdomain, tmodel)],
    bc=bc_fixed + bc_loaded,
)

In [7]:
s = ts.sim
s.tloop.k_max = 10
s.tline.step = 1
s.tloop.verbose = True
s.run()

t:	 0.00(0), 
	 1.00(1), 



In [8]:
U1 = ts.hist.U_t[-1]
print(U1)
eps1 = ts.fe_domain[0].xmodel.map_U_to_field(U1)
sig1, _ = ts.fe_domain[0].tmodel.get_corr_pred(eps1,1)
sig_x, sig_y, tau = sig1.T
sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float_))
sig_val, sig_dir = np.linalg.eig(sig_ab)
[np.min(sig_val), np.max(sig_val)]


[ 0.00000000e+00  0.00000000e+00  0.00000000e+00  5.88235294e-03
 -1.95156391e-18  0.00000000e+00  5.88235294e-03 -1.17647059e-03
  0.00000000e+00  0.00000000e+00 -1.17647059e-03  0.00000000e+00]


[-1.3877787807814457e-17, 0.20000000000000007]

In [9]:
sig1

array([[ 2.00000000e-01,  0.00000000e+00,  1.19996334e-17],
       [ 1.00000000e-01,  1.00000000e-01, -1.00000000e-01]])

In [10]:
sig_val

array([[ 2.00000000e-01, -7.19956011e-34],
       [-1.38777878e-17,  2.00000000e-01]])

In [11]:
eps1

array([[ 5.88235294e-06, -1.17647059e-06,  8.47032947e-22],
       [ 2.35294118e-06,  2.35294118e-06, -7.05882353e-06]])

In [12]:
3/28000

0.00010714285714285714